# Tools and Structured Output

The ability of LLMs to respond in structured format is helpful for a variety of use cases such as when we want to call funtions with rigid parameters or update databases with defined schemas.

In graphs that pass state variables between nodes there are a few different ways we can utilize tools and structured outputs. 

## Using ToolNode

In some cases, you might want to directly execute a tool node after calling your LLM. Perhaps you have made a weather chatbot such that whenever a question is asked, you want to immediately call a tool to pull the latest weather data. 

In this case, we can take  advantage of the built in [`ToolNode`](https://langchain-ai.github.io/langgraph/reference/prebuilt/#toolnode) functionality, and let Langchain do most of the work for us. By using `ToolNode` we can create a node that will automatically call the tool that was referenced in the last message in the chain. In our example we are creating a `ToolNode` with just a single tool, but you can pass multiple tools and the node will automatically select which one to execute.

In [67]:
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)

Just defining our tool and creating a corresponding `ToolNode` isn't enough - we need to make our LLM aware that it has the option to call such tools. We can do this by using the `bind_tools` method. In our example we actually want to force our LLM to call a specific tool, so we can use the `tool_choice` param to ensure this always happens.

In [84]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model_with_tools = model.bind_tools(tools,tool_choice='tavily_search_results_json')

We can now define the State class we will use for our graph, which in our example will just contain one member variables called `messages` which is a list of all the messages generated by our chain.

In [80]:
from typing import TypedDict, Annotated, Literal

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

We can now define the function for our model node and the layout of our graph. Since we are forcing it to call the tool - we don't need to define a routing function, we can just add an edge between our model node and our tool node. After calling our tool, we want to route to another LLM node which will return a plain text response to our user.

In [81]:
from langgraph.graph import StateGraph, END

def call_model_with_tools(state: AgentState) -> AgentState:
    return {"messages":[model_with_tools.invoke(state['messages'])]}

def call_model(state: AgentState) -> AgentState:
    return {"messages":[model.invoke(state['messages'])]}

graph = StateGraph(AgentState)
graph.add_node("agent", call_model_with_tools)
graph.add_node("respond_to_search", call_model)
graph.add_node("search", tool_node)
graph.set_entry_point("agent")
graph.add_edge("agent","search")
graph.add_edge("search","respond_to_search")
graph.add_edge("respond_to_search",END)
app = graph.compile()

Let's invoke our graph and see what happens.

In [82]:
from langchain_core.messages import HumanMessage

ans = app.invoke({'messages':[HumanMessage(content="What is the weather in Barcelona?")]})
for message in ans['messages']:
    message.pretty_print()

================================ Human Message =================================

What is the weather in Barcelona?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_IrjsLY2IuxvVs9K6PHxBFkjm)
 Call ID: call_IrjsLY2IuxvVs9K6PHxBFkjm
  Args:
    query: weather in Barcelona
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Barcelona', 'region': 'Catalonia', 'country': 'Spain', 'lat': 41.38, 'lon': 2.18, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1718211272, 'localtime': '2024-06-12 18:54'}, 'current': {'last_updated_epoch': 1718210700, 'last_updated': '2024-06-12 18:45', 'temp_c': 21.1, 'temp_f': 70.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree

In this example we used three nodes, but you could simplify your graph to just use two nodes and a routing function that determines whether to proceed from the LLM node to the tool node. Later in this page we will show examples of how to use routing functions with tool calls.

## Customize the tool node

Expanding on the above example, we can customize our tool nodes in many ways. One way we might customize our tool node is to update the state within the tool node, or pass in additional parameters to our tool node that come from sources other than the LLM.

Let's examine how we could customize the parameters we pass to our tool node. In our case we will customize the `max_results` param we pass to Tavily using user input, but we could also augment the LLM output with values from our state, values from other tool nodes, etc.

In [88]:
from langchain.tools import tool
from langgraph.prebuilt import ToolExecutor

@tool
def search(query: str, max_results: int):
    '''Look things up online'''
    return [str(TavilySearchResults(max_results=max_results).invoke(query)).replace('\'','\"')]


tools = [search]
tool_executor = ToolExecutor(tools)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model_with_tools = model.bind_tools(tools)

Instead of using the built in `ToolNode` functionality, we can now define a custom function for our tool node that will ask the user for how many search results they want returned, and then call our `search` tool with the right input.

In [89]:
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import ToolMessage

def execute_search_with_user_input(state: AgentState):
    max_results = input(prompt="How many search results should we return? Enter a number 1-10")

    if max_results.isdigit() and int(max_results) > 0 and int(max_results) <= 10:
        last_message = state['messages'][-1]
        tool_call = last_message.tool_calls[0]
        action = ToolInvocation(
            tool=tool_call["name"],
            tool_input={**tool_call["args"],**{'max_results':int(max_results)}},
        )
        response = tool_executor.invoke(action)
        return {'messages':[ToolMessage(content=str(response),tool_call_id=tool_call['id'])]}
    else:
        raise ValueError

We can define our graph almost identically to the one above, and run it again to examine the output.

In [90]:
graph = StateGraph(AgentState)
graph.add_node("agent", call_model_with_tools)
graph.add_node("respond_to_search", call_model)
graph.add_node("search", execute_search_with_user_input)
graph.set_entry_point("agent")
graph.add_edge("agent","search")
graph.add_edge("search","respond_to_search")
graph.add_edge("respond_to_search",END)
app = graph.compile()

In [91]:
ans = app.invoke({'messages':[HumanMessage(content="What is the weather in Barcelona?")]})
for message in ans['messages']:
    message.pretty_print()

================================ Human Message =================================

What is the weather in Barcelona?
================================== Ai Message ==================================
Tool Calls:
  search (call_H82UfLj030KPvA1PoAxa8G2o)
 Call ID: call_H82UfLj030KPvA1PoAxa8G2o
  Args:
    query: weather in Barcelona
    max_results: 1
================================= Tool Message =================================

['[{"url": "https://www.weatherapi.com/", "content": "{"location": {"name": "Barcelona", "region": "Catalonia", "country": "Spain", "lat": 41.38, "lon": 2.18, "tz_id": "Europe/Madrid", "localtime_epoch": 1718212317, "localtime": "2024-06-12 19:11"}, "current": {"last_updated_epoch": 1718211600, "last_updated": "2024-06-12 19:00", "temp_c": 19.4, "temp_f": 66.9, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 6.9, "wind_kph": 11.2, "wind_degree": 110, "wind_dir": "ESE", "press

As you can see the AI message passed a `max_results` value of 1, but by allowing the user to input a different value (in this case I entered 4), we can customize how the tool gets called.

Another way we could customize our tool node is to expand its functionality beyond just executing a tool call and have it update the graph state directly. To do this, let's add another field to our graph state called `urls` which will just keep track of all the URLs our search tool returns. This could be useful if we perhaps wanted to show the user where the LLM sourced its information from.

In [116]:
def add_urls(old_urls, new_urls):
    # Add URL lists together, skipping duplicates
    for url in new_urls:
        if url not in old_urls:
            old_urls.append(url)
    return old_urls

class AgentStateComplex(TypedDict):
    messages: Annotated[list, add_messages]
    urls: Annotated[list, add_urls]

We can now redefine our tool node to update the state when called.

In [126]:
import json

def execute_search_and_update_state(state: AgentState):

    last_message = state['messages'][-1]
    tool_call = last_message.tool_calls[0]
    action = ToolInvocation(
        tool=tool_call["name"],
        tool_input={**tool_call["args"],**{'max_results':3}},
    )
    response = tool_executor.invoke(action)
    urls = [search_response['url'] for search_response in json.loads(response[0].replace("\"{","{").replace("}\"","}"))]
    return {'messages':[ToolMessage(content=str(response),tool_call_id=tool_call['id'])],"urls":urls}

Our graph is defined identically to the example above, with a substitution for the tool node function.

In [127]:
graph = StateGraph(AgentStateComplex)
graph.add_node("agent", call_model_with_tools)
graph.add_node("respond_to_search", call_model)
graph.add_node("search", execute_search_and_update_state)
graph.set_entry_point("agent")
graph.add_edge("agent","search")
graph.add_edge("search","respond_to_search")
graph.add_edge("respond_to_search",END)
app = graph.compile()

In [130]:
ans = app.invoke({'messages':[HumanMessage(content="What is the weather in Barcelona?")]})
ans['urls']

['https://www.weatherapi.com/',
 'https://www.timeanddate.com/weather/spain/barcelona/ext',
 'https://www.bbc.com/weather/3128760']

We see that our tool node correctly updated the state, allowing us to store the URLs it used in creating it's response.

These are just two examples of how to customize your tool node, and there are more options available to developers such as streaming nested inputs within a tool node, nesting further tool calls within a single tool node, etc.

## Customize Routing and Output

In the examples above, we have been adding "strict" edges, in the sense that the LLM always proceeds automatically to calling the tool, which executes a single function. But this doesn't always need to be the case.

In fact, tools don’t have to be single functions, they are just ways for an LLM to generate structured output that your application can use. How it uses it is up to you. Some tools may not even correspond with a single scoped “tool” - they can be used for routing or for responding in a structured format.

Let's examine how we can use tools to have our graph return data in a structured format. We can achieve this by using Pydantic models. Let's continue with the weather chatbot example and define our structured output using the `BaseModel` and `Field` classes from Pydantic.

In [131]:
from langchain_core.pydantic_v1 import BaseModel, Field

# The docstring tells our model that formatting should be called as the final response to the user
class WeatherResponse(BaseModel):
    """Final response to the user"""

    temperature: float = Field(description="the temperature")
    wind_speed: float = Field(description="the wind speed")
    wind_direction: str = Field(description="the direction of the wind")

We can now create an LLM that uses `with_strctured_output` to return the result we would like to the user.

In [139]:
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults()]
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model_with_tools = model.bind_tools(tools,tool_choice='tavily_search_results_json')
model_with_structured_output = model.with_structured_output(WeatherResponse)
tool_node = ToolNode(tools)

def call_structured_model(state: AgentState) -> AgentState:
    return {"messages":[model_with_structured_output.invoke(state['messages'])]}

Our graph definition is again almost identical to the ones above.

In [140]:
graph = StateGraph(AgentState)
graph.add_node("agent", call_model_with_tools)
graph.add_node("respond_to_search_structured", call_structured_model)
graph.add_node("search", tool_node)
graph.set_entry_point("agent")
graph.add_edge("agent","search")
graph.add_edge("search","respond_to_search_structured")
graph.add_edge("respond_to_search_structured",END)
app = graph.compile()

Let's see what happens when we run it!

In [143]:
ans = app.invoke({'messages':[HumanMessage(content="What is the weather in Barcelona?")]})
for message in ans['messages']:
    try:
        message.pretty_print()
    except:
        print("================================= WeatherReponse =================================")
        print(message)

================================ Human Message =================================

What is the weather in Barcelona?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_wSfe8i00QKlX0olftBkLDsdA)
 Call ID: call_wSfe8i00QKlX0olftBkLDsdA
  Args:
    query: weather in Barcelona
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Barcelona', 'region': 'Catalonia', 'country': 'Spain', 'lat': 41.38, 'lon': 2.18, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1718214896, 'localtime': '2024-06-12 19:54'}, 'current': {'last_updated_epoch': 1718214300, 'last_updated': '2024-06-12 19:45', 'temp_c': 21.3, 'temp_f': 70.3, 'is_day': 1, 'condition': {'text': 'Moderate or heavy rain with thunder', 'icon': '//cdn.weatherapi.com/weather/64x64/day/389.png', 'code': 1276}, 'wind_mph': 10.5, 'wind_kp

Fantastic! Our graph returned data in the expected format, and with what looks like the correct values.

Let's now turn to using tools to make decisions in our graph, such as what node to proceed to next. To do this, we can define a pydantic model that will inform our node selection.

In [151]:
class RoutingFunction(BaseModel):
    """Select the next node to proceed to"""

    next_node: str = Field(description="The next node to travel to. The options are: node_2 or node_3")

tool_node = ToolNode([RoutingFunction])

Let's bind this "function" to our model and force it to select the tool, by passing in the param `tool_choice`.

In [152]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model = model.bind_tools([RoutingFunction],tool_choice="RoutingFunction")

Now let's define our new graph, first specifying the functions we will need for each of our nodes. In this example we are just testing whether our routing tool works as expected.

In [153]:
from langchain_core.messages import SystemMessage

def node_2_func(state):
    return {"messages":[SystemMessage(content=f"I made it to node 2!")]}

def node_3_func(state):
    return {"messages":[SystemMessage(content=f"I made it to node 3!")]}

We can now define our node functions and our graph.

In [154]:
def select_which_node(state: AgentState):
    return state['messages'][-1].content.split('\'')[1]

def call_model(state: AgentState) -> AgentState:
    return {"messages":[model.invoke(state['messages'])]}

In [155]:
graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("routing_tool", tool_node)
graph.add_edge("agent","routing_tool")
graph.add_node("node_2", node_2_func)
graph.add_node("node_3", node_3_func)
graph.set_entry_point("agent")
graph.add_conditional_edges("routing_tool",select_which_node)
graph.add_edge("node_2",END)
graph.add_edge("node_3",END)
app = graph.compile()

In [156]:
app.invoke({"messages":[HumanMessage(content="Please go to the node with the higher number")]})

{'messages': [HumanMessage(content='Please go to the node with the higher number'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4tvqTghlNlCeBpyvxYj51Af2', 'function': {'arguments': '{"next_node":"node_3"}', 'name': 'RoutingFunction'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'stop'}, id='run-829f4fc0-4490-4d59-80b0-ac9c6332fd6c-0', tool_calls=[{'name': 'RoutingFunction', 'args': {'next_node': 'node_3'}, 'id': 'call_4tvqTghlNlCeBpyvxYj51Af2'}]),
  ToolMessage(content="next_node='node_3'", name='RoutingFunction', tool_call_id='call_4tvqTghlNlCeBpyvxYj51Af2'),
  SystemMessage(content='I made it to node 3!')]}

As we can see, by using structured output we can select the next node to travel to without having to explicitly mention it's name.

## More to explore!

Hopefully this example doc gave you a good idea of a few of the ways to utilize tools and structured output in your Langgraph application. This list is neither exhaustive or mutually exclusive and there are a many more exciting ways you can customize how you use tools and structured output. We look forward to seeing all the creative ways developers like you utilize this exciting Langgraph functionality!